In [1]:
import sklearn.linear_model
import numpy as np
import pandas as pd
import os
import sys
sys.path.append(os.getcwd().split("src")[0])
from src.SoundLights.dataset.features_groups import  ARAUS_features, Freesound_features, mix_features, masker_features
from src.SoundLights.dataset.dataset_functions import prepare_data_models
from src.SoundLights.models.models_functions import clip, normalize_columns, normalize_columns_minmax

from sklearn.preprocessing import RobustScaler
import numpy as np


In [2]:
responses_Freesound= pd.read_csv(os.path.join('..','data/csv_files','SoundLights_Freesound.csv')) #, dtype = {'participant':str}
responses_Freesound, features=prepare_data_models(responses_Freesound, Freesound_features)
print(responses_Freesound)


       info.fold  info.stimulus_index  info.wav_gain  info.time_taken  \
0            0.0                 27.4           6.44          48.5390   
1            0.0                 19.4           6.44          58.2362   
2            0.0                 28.4           6.44          45.8852   
3            0.0                 26.0           6.44          48.5338   
4            0.0                 17.4           6.44          53.0804   
...          ...                  ...            ...              ...   
25243        5.0                 40.0           6.44         101.2900   
25244        5.0                 41.0           6.44          63.4210   
25245        5.0                 42.0           6.44          83.9920   
25246        5.0                 43.0           6.44          74.0230   
25247        5.0                 44.0           6.44          62.8560   

       info.is_attention  info.pleasant  info.eventful  info.chaotic  \
0                    0.0            2.8            

### Adjust alpha

In [3]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] 
l1_ratio = 0.5
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')

for value in alpha:

    model = ElasticNet(alpha=value, l1_ratio=l1_ratio, selection="random")
    #print(f'Investigating performance of {model} model...')

    MSEs_train = []
    MSEs_val = []
    MSEs_test = []
    MEs_train = []
    MEs_val = []
    MEs_test = []

    
    for val_fold in [1,2,3,4,5]:
        
        # Extract dataframes
        df_train = responses_Freesound[(responses_Freesound['info.fold'] != val_fold) & (responses_Freesound['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
        df_val   = responses_Freesound[responses_Freesound['info.fold'] == val_fold]
        df_test  = responses_Freesound[responses_Freesound['info.fold'] == 0] 

        # Get ground-truth labels
        Y_train = df_train['info.P_ground_truth'].values
        Y_val = df_val['info.P_ground_truth'].values
        Y_test = df_test['info.P_ground_truth'].values


        # Get features normalized_data = (data - min) / (max-min)
        X_train = df_train[features].values
        X_val =df_val[features].values
        X_test = df_test[features].values

        """ X_train, mean, std=normalize_columns(X_train)
        X_val= (X_val - mean) / (std)
        X_test= (X_test - mean) / (std) """
        X_train, min, max=normalize_columns_minmax(X_train)
        X_val= (X_val - min) / (max - min)
        X_test= (X_test - min) / (max - min)

        # Fit model
        X_LR = model.fit(X_train, Y_train)

        # Get MSEs
        MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
        MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
        MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
        ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
        ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
        ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

        # Add metrics
        MSEs_train.append(MSE_train)
        MSEs_val.append(MSE_val)
        MSEs_test.append(MSE_test)
        MEs_train.append(ME_train)
        MEs_val.append(ME_val)
        MEs_test.append(ME_test)

        #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
    print("Parameters ",value, l1_ratio )
    print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  0.1 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  0.2 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  0.3 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  0.4 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  0.5 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  0.6 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  0.7 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
P

### Adjust l1_ratio

In [5]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = 1
l1_ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] #0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')

for value in l1_ratio:

    model = ElasticNet(alpha=alpha, l1_ratio=value, selection="random")
    #print(f'Investigating performance of {model} model...')

    MSEs_train = []
    MSEs_val = []
    MSEs_test = []
    MEs_train = []
    MEs_val = []
    MEs_test = []

    
    for val_fold in [1,2,3,4,5]:

        # Extract dataframes
        df_train = responses_Freesound[(responses_Freesound['info.fold'] != val_fold) & (responses_Freesound['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
        df_val   = responses_Freesound[responses_Freesound['info.fold'] == val_fold]
        df_test  = responses_Freesound[responses_Freesound['info.fold'] == 0] # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

        # Get ground-truth labels
        Y_train = df_train['info.P_ground_truth'].values
        Y_val = df_val['info.P_ground_truth'].values
        Y_test = df_test['info.P_ground_truth'].values

        # Get features normalized_data = (data - min) / (max-min)
        X_train = df_train[features].values
        X_val =df_val[features].values
        X_test = df_test[features].values

        """ X_train, mean, std=normalize_columns(X_train)
        X_val= (X_val - mean) / (std)
        X_test= (X_test - mean) / (std) """
        X_train, min, max=normalize_columns_minmax(X_train)
        X_val= (X_val - min) / (max - min)
        X_test= (X_test - min) / (max - min)

        # Fit model
        X_LR = model.fit(X_train, Y_train)

        # Get MSEs
        MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
        MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
        MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
        ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
        ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
        ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

        # Add metrics
        MSEs_train.append(MSE_train)
        MSEs_val.append(MSE_val)
        MSEs_test.append(MSE_test)
        MEs_train.append(ME_train)
        MEs_val.append(ME_val)
        MEs_test.append(ME_test)

        #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
    print("Parameters ",alpha, value )
    print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')
    """ coefficients = X_LR.coef_
    feature_importances = list(zip(ARAUS_features+masker_features, coefficients))
    feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)
    list_features=[]
    for feature, importance in feature_importances:
        if float(importance)!=0:
            print(f"{feature}: {importance}")
            list_features.append(feature)
    print("LIST ",list_features) """

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  1 0.1
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  1 0.2
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  1 0.3
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  1 0.4
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  1 0.5
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  1 0.6
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  1 0.7
Mean | 0.1551 | 0.1553 | 0.1192 | 0.3275 | 0.3277 | 0.2867 |
Parameters  1 0

In [28]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = 0.2
l1_ratio = 0.5
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')



model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, selection="random")
#print(f'Investigating performance of {model} model...')

MSEs_train = []
MSEs_val = []
MSEs_test = []
MEs_train = []
MEs_val = []
MEs_test = []


for val_fold in [1,2,3,4,5]:

    # Extract dataframes
    df_train = responses_Freesound[(responses_Freesound['info.fold'] != val_fold) & (responses_Freesound['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
    df_val   = responses_Freesound[responses_Freesound['info.fold'] == val_fold]
    df_test  = responses_Freesound[responses_Freesound['info.fold'] == 0]

    # Get ground-truth labels
    Y_train = df_train['info.P_ground_truth'].values
    Y_val = df_val['info.P_ground_truth'].values
    Y_test = df_test['info.P_ground_truth'].values

    # Get features normalized_data = (data - min) / (max-min)
    X_train = df_train[Freesound_features+masker_features].values
    X_val =df_val[Freesound_features+masker_features].values
    X_test = df_test[Freesound_features+masker_features].values

    # Fit model
    X_LR = model.fit(X_train, Y_train)

    # Get MSEs
    MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
    MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
    MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
    ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
    ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
    ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

    # Add metrics
    MSEs_train.append(MSE_train)
    MSEs_val.append(MSE_val)
    MSEs_test.append(MSE_test)
    MEs_train.append(ME_train)
    MEs_val.append(ME_val)
    MEs_test.append(ME_test)

    #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
print("Parameters ",alpha, l1_ratio )
print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')
coefficients = X_LR.coef_
feature_importances = list(zip(Freesound_features+masker_features, coefficients))
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)
list_features=[]
for feature, importance in feature_importances:
    if float(importance)!=0:
        print(f"{feature}: {importance}")
        list_features.append(feature)
print("LIST ",list_features)

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  0.2 0.6
Mean | 0.1302 | 0.1345 | 0.0768 | 0.2937 | 0.2983 | 0.2237 |
freesound.lowlevel.mfcc.avg.c0: -0.0014348699721731723
freesound.lowlevel.mfcc.p20.c4: -0.0011049539651674534
freesound.lowlevel.mfcc.p20.c5: -0.0009046960757058929
freesound.lowlevel.spectral_centroid.avg: 0.0005408350247687239
freesound.lowlevel.mfcc.p20.c0: -0.0004976780274350692
freesound.lowlevel.mfcc.var.c6: 0.0003939124652250017
freesound.lowlevel.spectral_kurtosis.var: -0.00026971734310292134
freesound.lowlevel.mfcc.p80.c1: -0.0002387269752865223
freesound.lowlevel.spectral_